### 라이브러리

In [18]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib 
matplotlib.rcParams['axes.unicode_minus'] = False
matplotlib.rcParams['font.family'] = "AppleGothic"

from sklearn.ensemble import RandomForestRegressor

from pycaret.regression import *
import catboost as cb

### 데이터 처리

In [19]:
train = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/train.csv')
test = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/test.csv')
submission = pd.read_csv('G:\내 드라이브\Github\Dacon_Prediction-of-number-AI/sample_submission.csv')

In [20]:
# 요일별 랭크를 부여하여 설정하였다.
weekdayrankfordinner = {
    '월' : 1,
    '화' : 2,
    '수' : 4,
    '목' : 3,
    '금' : 5
} 

train['요일(석식)'] = train['요일'].map(weekdayrankfordinner)
test['요일(석식)'] = test['요일'].map(weekdayrankfordinner)

In [21]:
train['일자'] = pd.to_datetime(train['일자'])
test['일자'] = pd.to_datetime(test['일자'])

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day
train['요일'] = train['일자'].dt.weekday


test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day
test['요일'] = test['일자'].dt.weekday

In [22]:
# 메뉴는 사용하지 않았다. 드랍

drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [23]:
# 본사정원수 변수 대신에 식사가능자수 변수를 만들어 사용하였다. 

train['식사가능자수'] = (train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수'])
test['식사가능자수'] = (test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수'])

### 중식 월 랭크

```
train.query('년 == 2020').groupby('월').mean()['중식계'].sort_values()

중식 월
6      760.000000
1      811.000000
5      815.578947
8      825.100000
7      827.826087
10     852.684211
11     864.700000
9      871.650000
4      924.650000
2      937.842105
3      958.090909
12    1048.500000
Name: 중식계, dtype: float64

2021년의 1월은 약 1000. 1월을 2021년꺼를 사용한 이유는 2월부터 재택근무를 시작하여서 2020년 2월 이전과 이후의 양상이 다르다고 판단하였기에.
```

### 석식 월 랭크
```
train.groupby('월').mean()['석식계'].sort_values()

월
12    401.168317
1     424.564356
11    434.250000
7     437.300000
5     458.131313
8     459.485981
6     460.355556
9     463.617021
4     480.825243
10    481.641304
2     514.860215
3     530.700935
Name: 석식계, dtype: float64

석식의 경우는 전체에서 월별 평균순으로 랭크를 매겼음.
```

In [24]:
# 자기개발의 날 제거한 중식계의 월을 랭크매긴다. 
# (1월은 2021년의 1월, 2~12월은 2020년의 동일월을 기준으로함.)

month1 = {
    6:12,
	5:11,
	8:10,
	7:9	,
	10:8,
	11:7,
	9:6,
	4:5,
	2:4,
	3:3,
	1:2,
	12:1
}


# 석식계의 월을 랭크매긴다.
month2 = {
    1:11,
    2:2,
    3:1,
    4:4,
    5:8,
    6:6,
    7:9,
    8:7,
    9:5,
    10:3,
    11:10,
    12:12
}
train['월(중식)'] = train['월'].map(month1)
test['월(중식)'] = test['월'].map(month1)

train['월(석식)'] = train['월'].map(month2)
test['월(석식)'] = test['월'].map(month2)

train['월(중식)'] = train['월(중식)'].astype('int') #int로 변경
test['월(중식)'] = test['월(중식)'].astype('int')
train['월(석식)'] = train['월(석식)'].astype('int')
test['월(석식)'] = test['월(석식)'].astype('int')

### 공휴일전후 변수 사용
공휴일 전은 수가 적고, 공휴일 후는 많은 양상을 보였다.
그 중 수가 적은 공휴일만 사용함. (후의 경우도 수가 적은 양상을 보이는 값은 같이 추가하였음)
2의 경우는 황금연휴라 생각되는 부분을 적음

In [25]:
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [26]:
train['공휴일전후'][4] = 1 #2
train['공휴일전후'][17] = 1 #
#train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
#train['공휴일전후'][222] = 1
train['공휴일전후'][224] = 1
train['공휴일전후'][225] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 2
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 2
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 2
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 2
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 1
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 2
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] =2
test['공휴일전후'][20] = 1 # 더미코딩시 test에 공휴일전후_1이 생기도록 하기위해 잠시 설정후 0으로 바꿔줌 

In [27]:
# 원핫인코딩 사용
train = pd.get_dummies(train, columns=['공휴일전후'])
test = pd.get_dummies(test, columns=['공휴일전후'])

test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0 # 다시 바꿔주었다.

In [28]:
train[train.석식계>0]['석식계'].mean() # 0이아닌값 평균

478.8605851979346

In [29]:
# 코로나 이전에 자기개발의 날을 월말 수요일에 시행하였으나,  코로나 발생이후 2020년에는 2월과 11월 단 2회밖에
# 시행되지않음. 이에따라 테스트셋에서도 시행이 되지 않을것이라 판단하였다
# 하지만 모델에서는 있다고 판단하여 1/27이 100 단위가 나와서, 이를 막기위해 1월의 자기개발의 날에는
# 0이 아닌 값들의 석식계 평균을 대입해주었다

# 현재 1월만 대체
train['석식계'][244] = 478.8605851979346
#train['석식계'][281] = 398
train['석식계'][492] = 478.8605851979346
#train['석식계'][502] = 398
#train['석식계'][510] = 398
#train['석식계'][529] = 398
train['석식계'][730] = 478.8605851979346
#train['석식계'][747] = 398
#train['석식계'][766] = 398
train['석식계'][973] = 478.8605851979346
#train['석식계'][993] = 398

In [30]:
# 금요일이 랭크5라서 석식계가 train의 평균보다 높게 나오는 경향이 있어 train의 금요일을 10씩 낮추어주었다.
train.loc[(train.요일==4), '석식계'] -= 10

### feature 선택

In [31]:
from catboost import CatBoostRegressor
cat = CatBoostRegressor()

In [32]:
# train = train[train.석식계!= 0]

x1_train = train[['요일', '본사시간외근무명령서승인건수', '식사가능자수','월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #중식계
x2_train = train[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수','월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']] #석식계

y1_train = train['중식계']
y2_train = train['석식계']

x1_test = test[['요일', '본사시간외근무명령서승인건수', '식사가능자수', '월(중식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]
x2_test = test[['요일(석식)', '본사시간외근무명령서승인건수', '식사가능자수', '월(석식)', '일','공휴일전후_0','공휴일전후_1','공휴일전후_2','본사출장자수']]

In [33]:
model1 = cat
model2 = cat

model1.fit(x1_train, y1_train) #중식계
pred1 = model1.predict(x1_test)

model2.fit(x2_train, y2_train) #석식계
pred2 = model2.predict(x2_test)

submission['중식계'] = pred1
submission['석식계'] = pred2

Learning rate set to 0.040347
0:	learn: 203.7657572	total: 3.92ms	remaining: 3.92s
1:	learn: 199.0244251	total: 5.53ms	remaining: 2.76s
2:	learn: 194.0322793	total: 7.14ms	remaining: 2.37s
3:	learn: 189.5959005	total: 8.34ms	remaining: 2.08s
4:	learn: 185.4566813	total: 10.2ms	remaining: 2.04s
5:	learn: 181.2524975	total: 11.9ms	remaining: 1.97s
6:	learn: 177.1912381	total: 13.6ms	remaining: 1.94s
7:	learn: 173.2319401	total: 15.3ms	remaining: 1.9s
8:	learn: 169.5132883	total: 16.9ms	remaining: 1.86s
9:	learn: 166.0075568	total: 18.6ms	remaining: 1.84s
10:	learn: 162.5962776	total: 20.2ms	remaining: 1.81s
11:	learn: 159.4383330	total: 21.8ms	remaining: 1.79s
12:	learn: 156.6007678	total: 23.8ms	remaining: 1.81s
13:	learn: 154.2442645	total: 25.4ms	remaining: 1.79s
14:	learn: 151.4360531	total: 27ms	remaining: 1.77s
15:	learn: 149.0920455	total: 28.6ms	remaining: 1.76s
16:	learn: 146.6930090	total: 30.2ms	remaining: 1.74s
17:	learn: 144.4868467	total: 31.7ms	remaining: 1.73s
18:	learn: 

In [34]:
submission

,일자,중식계,석식계
0,2021-01-27,1020.373486,321.318684
1,2021-01-28,936.881259,507.473362
2,2021-01-29,537.888905,233.021222
3,2021-02-01,1231.633778,518.737171
4,2021-02-02,1046.988826,569.793711
5,2021-02-03,1022.872828,515.775621
6,2021-02-04,998.936576,565.792804
7,2021-02-05,742.158380,369.800979
8,2021-02-08,1231.041688,683.592530
9,2021-02-09,1059.599943,585.701251


### export

In [17]:
submission.to_csv('submission.csv', index=False)

In [36]:
train.iloc[1,2] = 498